[![DOI](https://zenodo.org/badge/940091341.svg)](https://doi.org/10.5281/zenodo.14939868) <a href="https://www.globh2e.org.au/"><img src="https://img.shields.io/badge/ARC:Funding%20number-IC200100023-blue.svg"/></a>

# **Life cycle optimisation - Tutorial**
<!-- **<div style="text-align: left; font-size: 19px;"> Prospective life cycle assessment data generation**</div> -->

<div style="text-align: left; font-size: 16px;">Michaël Lejeune<sup>a,b</sup>, Sami Kara<sup>a,b</sup>, Michael Zwicky Hauschild<sup>c,d</sup>, Sareh Sharabifarahni<sup>a</sup>, Rahman Daiyan<sup>b,e</sup></div><br>

<div style="text-align: left; font-size: 13px;"><sup>a</sup>Sustainability in Manufacturing and Life Cycle Engineering Research Group, School of Mechanical and Manufacturing Engineering, the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>b</sup>Australian Research Council Training Centre for the Global Hydrogen Economy (GlobH2e), the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>c</sup>Centre for Absolute Sustainability, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>d</sup>Division for Quantitative Sustainability Assessment (QSA), Department of Environmental and Resource Engineering, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>e</sup>School of Minerals and Energy Engineering, The University of New South Wales, Sydney 2052, Australia</div><br>

<div style="text-align: left; font-size: 13px;"> Code developer and maintainer:<a href="mailto:m.lejeune@unsw.edu.au"> Michaël Lejeune</div></a><br>


# **1. Initialisation**

In [1]:
using JuMP, CPLEX,Distributions,lce,DataFrames,LinearAlgebra
using PyPlot
using SparseArrays
using JLD2
include("./tutorial/old/matrices.jl");


In [2]:
@load "../Source data/03_additional_data/1_00_total_human_impact/data_interaction_matrices.jld2" 𝚪ᵦ 𝚪ₕ catnames_ticks


3-element Vector{Symbol}:
 :𝚪ᵦ
 :𝚪ₕ
 :catnames_ticks

In [5]:
initProject("Natcoms")


┌ Info: Project Natcoms -> model:ecoinvent_3.9.1 RCP:0.0 SSP:0 year:0 loaded
└ @ lce /Users/mickael/Library/CloudStorage/OneDrive-UNSW/Research/Publications/Journal articles/1_Natcoms/code/lce/src/lce.jl:781


Project              Model          RCP      SSP      Year   Method        
──────────────────── ────────────── ──────── ──────── ────── ──────────────
Natcoms              ecoinvent_3.9.1 0.0      0             0 Planetary boundaries

Natcoms: ecoinvent_3.9.1 0.0 0 0 Planetary boundaries


# **2. Background**

Technosphere exchanges
$$
\boldsymbol{As=f}
\tag{1}
$$


In [6]:
hydro=getAct("electricity production, hydro, run-of-river","DE")


HTML{String}("<table style=\"border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; font-size: 12px;\">\n    <thead>\n        <!-- Top heading row with dark grey background and white bold font -->\n        <tr style=\"background-color: #415a77; font-weight: bold; color: White;\">\n            <td rowspan=\"2\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Activity key: <br><br>  17405\n            </td>\n            <td style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Activity name\n            </td>\n            <td colspan=\"3\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                electricity production, hydro, run-of-river, DE<br>\n            </td>\n        </tr>\n        <tr style=\"background-color: #415a77; font-weight: bold; color: white;\">\n            <td style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Reference flow\n            </td>\n            <td colspan=\"3\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                1.0 kilowatt hour of electricity, high voltage\n            </td>\n        </tr>\n        <td colspan=\"5\" style=\"border: 1px solid #ddd; padding: 7px; background-color: #8d99ae; font-weight: bold; color: White; text-align: center;\">Technosphere Exchanges</td>\n        <!-- Column names row with centered text -->\n        \n    </thead>\n    <tbody>\n        <tr style=\"font-weight: bold;background-color: #8d99ae; padding: 7px; color: White; text-align: center\">\n            <td style=\"border: 1px solid #ddd; background-color: #8d99ae; font-weight: bold; color: White; text-align: center;\">Exchange keys</td>\n\n            <td style=\"border: 1px solid #ddd;\">Reference flow</td>\n            <td style=\"border: 1px solid #ddd;\">Activity</td>\n            <td style=\"border: 1px solid #ddd;\">Amount</td>\n            <td style=\"border: 1px solid #ddd;\">Unit</td>\n        </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">5777</td>\n        <td style=\"border: 1px solid #ddd;\">hydropower plant, run-of-river</td>\n        <td style=\"border: 1px solid #ddd;\">hydropower plant construction, run-of-river - Europe without Switzerland</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">8.07284939292173e-13</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">unit</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">12204</td>\n        <td style=\"border: 1px solid #ddd;\">lubricating oil</td>\n        <td style=\"border: 1px solid #ddd;\">market for lubricating oil - RER</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">7.56e-6</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">kilogram</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">18780</td>\n        <td style=\"border: 1px solid #ddd;\">waste mineral oil</td>\n        <td style=\"border: 1px solid #ddd;\">market for waste mineral oil - Europe without Switzerland</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">-7.56e-6</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">kilogram</td>\n    </tr>\n<table style=\"border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; font-size: 12px;\">\n    <thead>    <tr style=\"font-weight: bold;\">\n        <td colspan=\"5\" style=\"border: 1px solid #ddd; padding: 7px; background-color: #344e41; color: White; text-align: center;\">Biosphere Exchanges</td>\n    </tr>\n<tr style=\"font-weight: bold; background-color: #588157; color: White; text-align: center\">\n\n    <td style=\"border: 1px solid #ddd; padding: 7px; font-weight: bold; color: White; text-align: center;\">Exchange keys</td> \n    <td style=\"border: 1px 

Reference flow               Activity                     Amount       Unit     Location         Key             
──────────────────────────── ──────────────────────────── ──────────── ──────── ──────────────── ────────────────
electricity, high voltage    electricity production, hydro, run-of-river          1.0 kilowatt hour DE               17405           

In [8]:
𝐀=Technosphere!().Matrix
𝐟 =zeros(size(𝐀,1))
𝐟[hydro.key]=1 # kWh


1

In [9]:
𝐬=𝐀\𝐟;


Biosphere exchanges (total inventory)

$$
\boldsymbol{Bs=BA^{-1}f=g}
\tag{2}
$$


In [10]:
𝐁=Biosphere!().Matrix
𝐠 =𝐁*𝐬;


Impact score calculation
$$
\boldsymbol{QBs=QBA^{-1}f=h}
\tag{3}
$$

In [11]:
𝐐=Characterisation!().Matrix
𝐡=𝐐*𝐠


10-element Vector{Float64}:
  1.4545328401119444e-15
  1.0969333415401025e-13
  3.367889117390341e-16
  6.090262474166107e-17
  4.050072668964943e-14
  3.877881919568896e-14
  8.266812952309373e-15
  5.00223248306664e-19
 -5.103531044483143e-19
  4.348251316080494e-15

Absolute environmental sustainability assessment calculation
$$
\boldsymbol{QBs\oslash\omega =QBA^{-1}f\oslash\omega =h\oslash\omega}
\tag{3}
$$

In [12]:
@load "./tutorial/data_SOS.jld2"


4-element Vector{Symbol}:
 :𝚫𝐗ᴾᵇ
 :𝚫𝐗ᴰ
 :control_var
 :units

In [13]:
α=2e-15 #allocation factor
𝛚=α*𝚫𝐗ᴾᵇ;


In [16]:
⊘ = ./;


In [17]:
𝐝 = 𝐡 ⊘ 𝛚


10-element Vector{Float64}:
  0.7272664200559721
  0.7617592649584044
  0.2447593835312747
  0.4350187481547218
  0.005062590836206179
  1.7626735998040437
  0.06666784638959172
  1.7865116010952285e-5
 -1.0207062088966285e-5
  0.21741256580402468

## **2.1 Life cycle optimisation models**

Kätelhön, Bardow and Suh, (2016)

$$
\min_{\text{s.t.}\ s} Z = \boldsymbol{k^T F s} 
\tag{1}
$$

$$
\boldsymbol{As=f} 
\tag{2}
$$


$$
\boldsymbol{s^*\geq 0} 
\tag{3}
$$

$$
\boldsymbol{F s \leq c} 
\tag{4}
$$

Bachmann et al. 2023

$$
\min_{\text{s.t.}\ s} PF_{CC} = \boldsymbol{q_{CC}^T B s} 
\tag{1}
$$

$$
\boldsymbol{As=f} 
\tag{2}
$$


$$
\boldsymbol{s\geq 0} 
\tag{3}
$$

$$
\boldsymbol{s \leq c} 
\tag{4}
$$
$$
\boldsymbol{qBs \leq PF_{z}} 
\tag{4}
$$

Letchemberg et al. 2024

$$
\min_{s, \text{slack}} w^T z
\tag{1}
$$

$$
A s = f + \text{slack}
\tag{2}
$$

$$
s^{\text{low}} \leq s \leq s^{\text{high}}
\tag{3}
$$

$$
0 \leq \text{slack} \leq \text{slack}^{\text{high}}
\tag{4}
$$

$$
z = Q B s
\tag{5}
$$

$$
z \leq z^{\text{high}}
\tag{6}
$$

$$
B s \leq b^{\text{high}}
\tag{7}
$$


In this research, we implement the following model

$$
\min_{\text{s.t.}\ s} \ \boldsymbol{x  = \Gamma d} 
\tag{1}
$$


$$
\boldsymbol{d = \left(\Lambda s\right)\oslash \omega = \left(Q B s\right)\oslash \omega}
\tag{2}
$$

$$
\boldsymbol{A s = f}
\tag{2}
$$

$$
\boldsymbol{s^* \geq 0}
\tag{2}
$$

$$
\boldsymbol{F s \leq c}
\tag{2}
$$

$$
\boldsymbol{x \leq x^{\text{high}}}
\tag{2}
$$

$$
\boldsymbol{Bs \leq g^{\text{high}}}
\tag{2}
$$


# Modelling


## **Safe operating space**

In [18]:
b = 1:length(processes) # here n refers to the columns of matrix A, which are the processes involved. 
α = 3.5e-12*0.5
ω = (α*𝚫𝐗ᴾᵇ)


10-element Vector{Float64}:
 1.75e-12
 1.26e-10
 1.2040000000000003e-12
 1.225e-13
 7.0e-9
 1.925e-11
 1.085e-10
 2.45e-11
 4.375e-11
 1.75e-11

In [ ]:
𝐌 = [3	0.25	0.25	0.25	0.25	0.25	0.25	0.25	0.25	0.25	0.25	0.25	0.25	0.25	0.01]


In [ ]:
model = Model(CPLEX.Optimizer)
set_silent(model)

# Variables and expressions
@variable(model, 𝐬[1:length(processes)])

@expression(model, 𝐝, Λ*𝐬 ⊘ ω)
# @expression(model, 𝐱, 𝚪ᵦ*𝐝)
@expression(model, 𝐱, 𝐝)
@expression(model, φ, 𝐌*𝐬)

@objective(model, Min, 𝐱[1]);
@constraint(model, 𝐬 ≥ 0)
@constraint(model, A * 𝐬 == f)
@constraint(model, 𝐬 ≤ c)

optimize!(model);
if !is_solved_and_feasible(model)
    error("Solver did not find an optimal solution")
end
# solution_summary(model)
DataFrame(hcat([catnames_ticks,value.(𝐝),value.(𝐱)]...),["Boundaries","𝐝", "𝐱"])


In [ ]:
value.(φ)


In [ ]:
value.(𝐬)


## **3.1 Optimisation**

In [ ]:
model = Model(CPLEX.Optimizer)
set_silent(model)

# Variables and expressions
@variable(model, 𝐬[1:length(processes)])

@expression(model, 𝐠, B*𝐬)
@expression(model, 𝐝, Q*𝐠 ⊘ ω)
@expression(model, 𝐱, 𝐝)

@objective(model, Min, 𝐱);
@constraint(model, 𝐬 ≥ 0)
@constraint(model, A * 𝐬 == f)
@constraint(model, F * 𝐬 ≤ c)
optimize!(model);
if !is_solved_and_feasible(model)
    error("Solver did not find an optimal solution")
end
# solution_summary(model)
DataFrame(hcat([catnames_ticks,value.(𝐝),value.(𝐱)]...),["Boundaries","𝐝", "𝐱"])


In [ ]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬]]...),["processes","𝐬"])


### **3.1.2 Climate change oriented optimisation**

In [ ]:
modelᶜᶜ = Model(CPLEX.Optimizer);
set_silent(modelᶜᶜ)

@variable(modelᶜᶜ, 0 ≤ 𝐬ᶜᶜ[1:length(processes)])

@expression(modelᶜᶜ, 𝐠, B*𝐬ᶜᶜ)
@expression(modelᶜᶜ, 𝐝ᶜᶜ, Q[1,:]'*𝐠./ω)
@expression(modelᶜᶜ, 𝐱ᶜᶜ, 𝐝ᶜᶜ)

@objective(modelᶜᶜ, Min, 𝐱ᶜᶜ)
@constraint(modelᶜᶜ , A * 𝐬ᶜᶜ == f) 
@constraint(modelᶜᶜ , F * 𝐬ᶜᶜ ≤ c)

# @constraint(modelᶜᶜ , ((Λ°[7,:]'*𝐬ᶜᶜ)/ω[7]) ≤ 6)

optimize!(modelᶜᶜ);
solution_summary(model)
𝐬°ᶜᶜ= value.(𝐬ᶜᶜ)
𝐝°ᶜᶜ = value.(𝐝ᶜᶜ)
𝐱°ᶜᶜ = value.(𝐱ᶜᶜ)
DataFrame(hcat([catnames_ticks,𝐝°ᶜᶜ,𝐱°ᶜᶜ]...),["Boundaries","𝐝ᶜᶜ", "𝐱ᶜᶜ"])


In [ ]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬ᶜᶜ]]...),["processes","𝐬"])


### **3.1.3 optimisation with informed by biophysical interactions**

#### **3.1.3.1 Regular optimisation**

In [ ]:
modelᵦ = Model(CPLEX.Optimizer)
set_silent(modelᵦ)

@variable(modelᵦ, 0 ≤ 𝐬ᵦ[1:length(processes)])

@expression(modelᵦ, 𝐠,  B*𝐬ᵦ)
@expression(modelᵦ, 𝐝ᵦ, Q*𝐠./ω)
@expression(modelᵦ, 𝐱ᵦ, 𝚪ᵦ * 𝐝ᵦ)

@constraint(modelᵦ, A * 𝐬ᵦ == f)
@constraint(modelᵦ, F * 𝐬ᵦ ≤ c)

@objective(modelᵦ, Min, 𝐱ᵦ)
optimize!(modelᵦ)
solution_summary(modelᵦ)

DataFrame(hcat(catnames_ticks,[value.(i) for i ∈ [𝐝ᵦ,𝐱ᵦ]]...),["Boundaries","𝐝ᵦ", "𝐱ᵦ"])


In [ ]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬ᵦ]]...),["processes","𝐬ᵦ"])


#### **3.1.3.2 Stochastic optimisation**

In [ ]:
# @load "../Source data/03_additional_data//1_00_total_human_impact/data_interaction_param_matrices.jld2" δ𝐁


In [ ]:
# σω = 0.001 .* abs.(ω) # standard deviation for the perturbed vector ω
# σB= 0.001 .* abs.(B) # standard deviation for the perturbed matrix B°
# σF= 0.001 .* abs.(F) # standard deviation for the perturbed matrix F°
# σA= 0.001 .* abs.(A) # standard deviation for the perturbed matrix A°
# σf = 0.001 .* abs.(f) # standard deviation for the perturbed vector f
# σc = 0.001 .* abs.(c) # standard deviation for the perturbed vector c

# N = 10000
# s_res=zeros(length(processes),N);
# d_res=zeros(length(boundaries),N);
# x_res=zeros(length(boundaries),N);


In [ ]:
# for k ∈ 1:N

#     modelᵦ = Model(CPLEX.Optimizer)
#     set_silent(modelᵦ)
#     𝚪ᵦ°= Matrix(𝐈-matformat(rand.(δ𝐁)))^-1 #FIXME: uncertainties
#     ω°= rand.(TriangularDist.(ω.*0.8, ω.*1.2,ω))
#     A°= rand.(Normal.(A, σA)) # A shouldn't change, 
#     B°= rand.(Normal.(B, σB))
#     F°= rand.(Normal.(F, σF))
#     f°= rand.(Normal.(f, σf))
#     c°= rand.(Normal.(c, σc))

#     # Decision variable
#     @variable(modelᵦ, 0 ≤ 𝐬ᵦ[1:length(processes)])

#     # Expressions (equalities)
#     @expression(modelᵦ, 𝐠ᵦ, B°*𝐬ᵦ)
#     @expression(modelᵦ, 𝐝ᵦ, (Q*𝐠ᵦ)./ω°)
#     @expression(modelᵦ, 𝐱ᵦ, 𝚪ᵦ * 𝐝ᵦ)

#     # Expressions (Constraints on the decision variable)
#     @constraint(modelᵦ, A° * 𝐬ᵦ == f°)
#     @constraint(modelᵦ, F° * 𝐬ᵦ ≤ c°)

#     # Objective function: Minimising the state of control variables
#     @objective(modelᵦ, Min, 𝐱ᵦ)    
#     optimize!(modelᵦ)
#     if !is_solved_and_feasible(modelᵦ)
#         error("Solver did not find an optimal solution")
#     end

#     # extract values to store stochastic results
#     d_res[:,k] = value.(𝐝ᵦ)
#     s_res[:,k] = value.(𝐬ᵦ)
#     x_res[:,k] = value.(𝐱ᵦ)
# end
# solution_summary(modelᵦ)


In [ ]:
# fig, axs = subplots(3,3,figsize=(10,10),sharey=true)
# for i in 1:9
#     axs[i].hist(x_res[i,:], bins=100, alpha=0.5)
#     axs[i].axvline(median(x_res[i,:]), color="red", linestyle="--", label="median")
#     axs[i].set_title(catnames_ticks[i])
#     axs[i].legend(frameon=false)
# end
# display(plt.gcf())


In [ ]:
fig, axs = subplots(3,3,figsize=(10,10))
for i in 1:9
    axs[i].hist(d_res[i,:], bins=100, alpha=0.5)
end
display(plt.gcf())


In [ ]:
fig, axs = subplots(4,4,figsize=(12,10))
for i in 1:9
    axs[i].hist(s_res[i,:], bins=100, alpha=0.5)
end
display(plt.gcf())


### **3.1.4 Optimisation with informed by biophysical+human-mediated interactions**

In [ ]:
modelₕ = Model(CPLEX.Optimizer);
set_silent(modelₕ)

@variable(modelₕ, 0≤ 𝐬ₕ[1:length(processes)])

@expression(modelₕ, 𝐠ₕ, B*𝐬ₕ)
@expression(modelₕ, 𝐝ₕ, (Q*𝐠ₕ)./ω)
@expression(modelₕ, 𝐱ₕ, 𝚪ₕ * 𝐝ₕ)

@objective(modelₕ, Min, 𝐱ₕ)
@constraint(modelₕ, A * 𝐬ₕ == f) 
@constraint(modelₕ, F * 𝐬ₕ ≤ c)
optimize!(modelₕ);
# solution_summary(modelₕ)
DataFrame(hcat(catnames_ticks,[value.(i) for i ∈ [𝐝ₕ,𝐱ₕ]]...),["Boundaries","𝐝ₕ", "𝐱ₕ"])


In [ ]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬ₕ]]...),["processes","𝐬"])


## **3.2. Results**

### **3.2.1 System analysis**

In [ ]:
S=[𝐬, 𝐬ᶜᶜ,𝐬ᵦ,𝐬ₕ]
DataFrame(hcat([processes,[value.(i) for i ∈ S]...]...),["processes","s", "sᶜᶜ", "sᵦ","sₕ"])



### **3.2.2 Direct human impact**

In [ ]:
S=[𝐝,𝐝ᶜᶜ,𝐝ᵦ,𝐝ₕ]
DataFrame(hcat([catnames_ticks,[value.(i) for i ∈ S]...]...),["processes","𝐝","𝐝ᶜᶜ","𝐝ᵦ","𝐝ₕ"])


### **3.2.3 State of plenatary bondaries**

In [ ]:
S=[𝐱,𝐱ᶜᶜ,𝐱ᵦ,𝐱ₕ]
DataFrame(hcat([catnames_ticks,[value.(i) for i ∈ S]...]...),["processes","𝐱","𝐱ᶜᶜ","𝐱ᵦ","𝐱ₕ"])
